#### Library installation used web link

In [1]:
# pip install numpy pandas matplotlib pyrebase4 python-firebase




## Google cloud console library 

In [1]:
import os
import schedule
import time
from googleapiclient.discovery import build
from google.oauth2 import service_account

## Firebase library

In [9]:
import pyrebase

config = {
    'apiKey': "****",
    'authDomain': "****",
    'databaseURL': "****",
    'projectId': "****",
    'storageBucket': "****",
    'messagingSenderId': "****",
    'appId': "****",
    'measurementId': "****"
}

from firebase import firebase
firebase = firebase.FirebaseApplication('****', None)
firebase = pyrebase.initialize_app(config)
db = firebase.database()
# users = db.child("elated-capsule-249919:Heart_rate/-MaLuG5cohlzZx_TLFl9").get()

### <u>Operation</u>

### 1. extract the data from firebase
### 2. push the data into spreadsheet using google cloud console
### 3. schedule the script at every ```3 seconds```

In [2]:
def a():
    condition_val = 0
    nu = 1
    num = 1
    counter_humid = 1
    counter_temp = 1
    bodytemp_count = 1
    c = 0
    
    doubly_arr = []
    time_period = []
    pulse = []
    humidity = []
    temperature = []
    body_temp = []
    
    users = db.child("test/push").get()
    
    if condition_val==0:
        time_period.insert(0,"Time")
        pulse.insert(0,"Pulse")
        humidity.insert(0,"Humidity")
        temperature.insert(0,"ENV. Temperature")
        body_temp.insert(0,"Body Temperature")
        condition_val=1
    
        
    for user in users.each():
        time_period.insert(num,list(user.val().values())[4])
        num+=1
    print(time_period)

    for user in users.each():
        pulse.insert(nu,list(user.val().values())[2])
        nu+=1
    print(pulse)
    
    
    for user in users.each():
        humidity.insert(counter_humid,list(user.val().values())[1])
        counter_humid+=1
    print(humidity)

    
    for user in users.each():
        temperature.insert(counter_temp,list(user.val().values())[3])
        counter_temp+=1
    print(temperature)
    
    for user in users.each():
        body_temp.insert(bodytemp_count,list(user.val().values())[0])
#         body_temp.insert(bodytemp_count,0)
        bodytemp_count+=1
    print(body_temp)
    
    for i in range(len(pulse)):
        doubly_arr.append([time_period[c],pulse[c],humidity[c],temperature[c],body_temp[c]])
#         doubly_arr.append([time_period[c],pulse[c]])
        c+=1
    
    send_alert = [['heart beat Alert'],['Body temperature Alert']]
    alert_bool_1 = int(pulse[-1])
    alert_bool_2 = int(body_temp[-1])
    alert = ""
    alert2 = ""
    if alert_bool_1<60:
        alert = 'Low Heartbeat ⚠️'
    elif alert_bool_1>120:
         alert = 'High Heartbeat ⚠️'
    else:
        alert = 'Normal Heartbeat 💚'
    send_alert[0].insert(1,alert)
    
    if alert_bool_2<=95:
        alert = 'Low temp. ⚠️'
    elif alert_bool_2>=106:
         alert = 'High temp. ⚠️'
    else:
        alert = 'Normal 💚'
    send_alert[1].insert(1,alert)
    
    
    SCOPES = ['****']
    SERVICE_ACCOUNT_FILE = 'keys.json'

    creds = None
    creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    # If modifying these scopes, delete the file token.json.
    # The ID spreadsheet.
    SAMPLE_SPREADSHEET_ID = '****'
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()

    # result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
    #                                 range="test!A1:B5000000").execute()

    aoa = doubly_arr

    update = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                   range="testing!A1", valueInputOption="USER_ENTERED", body={"values":aoa}).execute()
    
    alert_val = send_alert
    print(send_alert)

    update = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                   range="heart_agg!A2", valueInputOption="USER_ENTERED", body={"values":alert_val}).execute()

In [22]:
try:
    a()
except:
    print('connection failed')

['Time', 15, 33, 50, 68, 85, 102, 119, 140, 157, 174, 191, 210]
['Pulse', 0, 68, 76, 68, 76, 40, 84, 0, 88, 36, 64, 4]
['Humidity', 93, 92, 92, 92, 92, 92, 91, 91, 91, 91, 91, 91]
['ENV. Temperature', 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28]
['Body Temperature', 96.16016, 101.43359, 102.3125, 101.43359, 101.43359, 101.43359, 97.03906, 97.03906, 101.43359, 101.43359, 97.03906, 101.43359]
[['heart beat Alert', 'Low Heartbeat ⚠️'], ['Body temperature Alert', 'Normal 💚']]


# ```schedule``` the task

In [24]:
schedule.every(2).seconds.do(a)

while True:
    schedule.run_pending()
#     time.sleep(1)
    print("*************")